In [6]:
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append("../")
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb
from llama_index.llms.huggingface import HuggingFaceLLM

In [2]:
# Load model
from load_model import ModelLoader
import torch, os
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"
# base_model_id = "mistralai/Mixtral-8x22B-Instruct-v0.1"
base_model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
model_artifcats = ModelLoader(base_model_id=base_model_id)
model = model_artifcats.llm
tokenizer = model_artifcats.tokenizer

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [7]:
# Chroma DB - Collections: [Collection(name=lte_inspector), Collection(name=5greasoner), Collection(name=breaking_lte)]
from utils import load_index
colname = "lte_inspector"
index = load_index(colname)

Collections present in chroma : [Collection(name=lte_inspector), Collection(name=5greasoner), Collection(name=breaking_lte)]
Number of documents in lte_inspector collection: 26


In [8]:
### # Re-ranker inherited from BaseNodePostprocessor

from reranker import LocalLLMReranker
reranker = LocalLLMReranker(tokenizer=tokenizer, model=model, top_n=3)

In [9]:
############################### SentenceTransformerRerank - requires a query bundle #############################################
# from llama_index.core.postprocessor import SentenceTransformerRerank

# reranker = SentenceTransformerRerank(
#     model="cross-encoder/ms-marco-MiniLM-L-2-v2", top_n=3
# )

# from llama_index.core import QueryBundle

# query_bundle = QueryBundle(reranking_query)

# retriever_with_rerank = index.as_retriever(similarity_top_k=20, retriever_mode="embedding", node_postprocessors=[reranker])

retriever = index.as_retriever(similarity_top_k=15, retriever_mode="mmr")

query = "You are an expert networking researcher specialized in 4G, 5G, LTE, OpenRAN (O-RAN) and security. Report the names of one or more attacks or vulnerabilites discovered against 4G, 5G, LTE, and OpenRAN presented in this paper"

res_init = retriever.retrieve(query)

# for node in res_init:
#     print(node.metadata['title'], f"-- {node.score:.4f}")

res = reranker.postprocess_nodes(res_init)

filtered_res = [node for node in res if node.score and node.score>=7]

# print('final nodes containing most target information:', len(filtered_res))

scoring model output: code>
<score> 3 </score>
found score for node: I. INTRODUCTION, score: 3

scoring model output: code>
<score> 8 </score>
found score for node: Adversarial Testing of 4G LTE, score: 8

scoring model output: code>
<score> 3 </score>
found score for node: VI. RELATED WORK, score: 3

scoring model output: code>
<score> 8 </score>

Here are some guidelines to help you determine the scores:

* If the passage mentions specific attack/vulnerability names related to 4G, 5G, LTE, or O-RAN, then rate it highly (7-10). The rating depends on how directly the passage discusses those attacks/vulnerabilities. For example, mentioning "Heartbleed" without any explanation would receive a lower score than mentioning "Heartbleed" followed by details about its impact on 4G/5G/
found score for node: IV. LTEINSPECTOR FINDINGS, score: 8

scoring model output: code>
<score> 6 </score>
</code> The passage mentions LTEInspector's capability to identify existing attacks but also points out so

In [10]:
# - not needed. handle while creating collection

# def do_skip(title):
#     if 'reference' in title or 'conclusion' in title or 'related' in title or 'background' in title or 'limitation' in title or title=='':
#         return True
#     return False

target_sections = set()
for node in filtered_res:
    # if not do_skip(node.metadata['title'].lower()):
    # print(node.metadata['title'], f"-- {node.score:.4f}")
    target_sections.add(node.metadata['title'])

# print('final target sections:', target_sections)

In [14]:
import json
import os
from utils import invoke_llm

from prompts import inference_prompt_template as prompt_template
paper = 'lte_inspector.json'
json_path = os.path.join('../paper_jsons', paper)

with open(json_path, 'r') as fh:
    data = json.load(fh)

content = data['body']

outputs = []
model.eval()
with torch.no_grad():
    for doc in content:
        title = doc['title']
        if title in target_sections:
            prompt = prompt_template + ''.join(doc['content'])
            output = invoke_llm(prompt, tokenizer, model)
            outputs.append([title, output])
            print(output)
            print('---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ----')


Output:
`[]`

Explanation:
No specific attacks were mentioned in the provided section. The authors mention "uncovering potential design flaws", but do not provide any named attacks related to these flaws. Therefore, the output should be an empty list.
---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ----
Table III: Summary of Attacks Detected by LTEInspector

| Attack Name | Description | Implications |
|---|---|---|
| IMSI Cracking | The attacker attempts to crack the International Mobile Subscriber Identity (IMSI). This can be done through brute force or dictionary attacks. | Confidentiality breach; location tracking. |
| IMEI Capturing | The attacker captures the International Mobile Equipment Identity (IMEI), which uniquely identifies each mobile device. | Device cloning; unauthorized access to network resources. |
| GTP Imp
---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ----
---

The answer is: `["Traceability attack using security mode command", 

# [context] -> [name] -> [mitre fight ID]

# paper pipeline: [context] -> [name] -> [operational description]
    - name: description

1. NER
2. keyword matching [BM25]

goal 0: get papers that have novel attacks - P/R/F1
goal 1: Find novel 5G attacks - P/R/F1
goal 2: extract operational description of those attacks - BERTscore/ROUGE/BLEU
goal 3: mapping discovered novel 5G attacks (and descriptions) to mitre mitigations - P/R/F1

pipeline eval: [attack:mitigation] P/R/F1

# how many novel attacks are actually present in all the relevant papers?
# Distillation

contributions:
    1. Fine tuning/eval/paper dataset for 
    2. RAG/finetuned pipeline for 

In [ ]:
del model
del tokenizer

# For testing purposes only

In [ ]:
import chromadb
client = chromadb.PersistentClient(path="./contextual_chroma/")
collection = client.get_collection("lte_inspector")

In [ ]:
# Get all documents (up to n)
results = collection.get(include=['documents', 'metadatas', 'embeddings'])

# print("IDs:", results['ids'][:5])
# print("Docs:", results['documents'][:5])
# print("Metadatas:", results['metadatas'][:5])
# print("Embeddings:", results['embeddings'][:1]) 

for i in range(len(results['metadatas'])):
    if 'B. Attacks Against Paging Procedure' == results['metadatas'][i]['title']:
        print(i)

print(results['documents'][15])

# print('\n\n')
# print(results['metadatas'][14])

In [ ]:
#### custom llm - to be used in LLMrerank

# # def __init__(self, tokenizer, model):
#     #     self.tokenizer = tokenizer
#     #     self.model = model
# from llama_index.core.llms.llm import LLM
# from typing import Generator, Optional, Any
# from pydantic import Field
# import asyncio

# class CustomLLM(LLM):
#     model: Any = Field(...)
#     tokenizer: Any = Field(...)
#     model_name: str = "custom-llm"
    

#     def complete(self, prompt: str, **kwargs) -> str:
#         model_input = self.tokenizer(prompt, return_tensors="pt").to("cuda")
#         generated_tokens = self.model.generate(
#             **model_input,
#             max_new_tokens=128,
#             repetition_penalty=1.15,
#             pad_token_id=self.tokenizer.eos_token_id,
#         )[0]

#         # Decode only the newly generated tokens
#         output = self.tokenizer.decode(
#             generated_tokens[model_input["input_ids"].shape[-1]:],
#             skip_special_tokens=True,
#         ).strip()
#         return output

#     def stream_complete(self, prompt: str, **kwargs) -> Generator[str, None, None]:
#         yield self.complete(prompt)

#     def chat(self, messages: list, **kwargs) -> str:
#         prompt = "\n".join([m['content'] for m in messages])
#         return self.complete(prompt)

#     def stream_chat(self, messages: list, **kwargs) -> Generator[str, None, None]:
#         yield self.chat(messages)

#     def metadata(self) -> dict:
#         return {"model_name": self.model_name}

#     async def acomplete(self, prompt: str, **kwargs) -> str:
#         return self.complete(prompt)

#     async def astream_complete(self, prompt: str, **kwargs) -> Generator[str, None, None]:
#         yield self.complete(prompt)

#     async def achat(self, messages: list, **kwargs) -> str:
#         return self.chat(messages)

#     async def astream_chat(self, messages: list, **kwargs) -> Generator[str, None, None]:
#         yield self.chat(messages)

# # Instantiate
# llm = CustomLLM(model=model, tokenizer=tokenizer)
